In [ ]:
import pandas as pd
import itertools

In [ ]:
file = 'legacy_data/20180925_StGBl fertig.xlsx'
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
df = pd.read_excel(file).fillna('False')
archiv, _ = Institution.objects.get_or_create(
    written_name='Österreichische Nationalbibliothek',
    abbreviation="ÖNB",
    institution_type="Archiv"
)

In [ ]:
for i, row in df.iterrows():
    item = None
    signatur = ", ".join(
            [
                "Bestand: {}".format(row['Teil von Bestand']),
                "Signatur: {}".format(row['(Archiv)Signatur'])
            ]
    )
    try:
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
    except Exception as e:
        print(e)
    if item:
        print(signatur)
        if row['Schlagwörter normalisiert'] != 'False':
            slw = row['Schlagwörter normalisiert'].split(';')
            concepts = []
            for y in slw:
                concepts.append(list(SkosConcept.objects.filter(pref_label=y.strip())))
                concepts.append(list(SkosConcept.objects.filter(other_label__label=y.strip())))
            if concepts:
                item.subject_norm.set(list(itertools.chain.from_iterable(concepts)))
        if row['Titel der Rechtsnorm'] != 'False':
            item.title = row['Titel der Rechtsnorm']
        if row['Zusammenfassung'] != 'False':
            item.abstract = row['Zusammenfassung']
        if row['Anmerkungen'] != 'False':
            item.permalink = row['Anmerkungen']
        if row['Datum orig'] != 'False':
            item.written_date = row['Datum orig']
        if row['Typ des Dok'] != 'False':
            type_vocab, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(row['Typ des Dok'])
            )
            type_vocab.scheme.add(res_type_scheme)
            item.res_type = type_vocab
        if row['Datum normalisiert DD/MM/YYYY)\n'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Datum normalisiert DD/MM/YYYY)\n'])
            except Exception as e:
                print(e)
        item.archiv = archiv
        item.save()